# Multi-step tool calling: Manual first, then Automatic (Agent)

This notebook demonstrates manual multi-tool orchestration first, followed by automatic (agent) orchestration, using two tools.


1. Instantiate the client

In [1]:
import os, sys
from dotenv import load_dotenv
from envyaml import EnvYAML
from langchain.tools import tool
from langchain_core.messages import HumanMessage
from langchain.agents import create_agent

from langChain.oci_openai_helper import OCIOpenAIHelper
load_dotenv()

def load_config(config_path):
    try:
        with open(config_path, 'r') as f:
            return EnvYAML(config_path)
    except FileNotFoundError:
        print(f"Error: Configuration file '{config_path}' not found.")
        return None

def pretty_print(response):
    for i, m in enumerate(response["messages"], 1):
        role = getattr(m, "type", m.__class__.__name__)
        content = m.content if isinstance(m.content, str) else str(m.content)
        print(f"{i:>2}. [{role.upper()}] {content}")

SANDBOX_CONFIG_FILE = "sandbox.yaml"
LLM_MODEL = "openai.gpt-5"

scfg = load_config(SANDBOX_CONFIG_FILE)
llm = OCIOpenAIHelper.get_client(
    model_name=LLM_MODEL,
    config=scfg,
    use_responses_api=True,
    reasoning={"effort": "low", "summary": "auto"}
)


2. define the tools

In [2]:
from langchain.tools import tool

@tool
def get_weather(city: str) -> str:
    """Gets the weather for a given city"""
    return f"The weather in {city} is 70 Fahrenheit"

@tool
def get_projection_bill(current_bill: int, gas_oven: bool) -> int:
    """Returns the projected bill depending on current bill and oven usage"""
    if gas_oven:
        return current_bill + 45
    return current_bill + 4

tools = [get_weather, get_projection_bill]
tool_map = {t.name.lower(): t for t in tools}


## Manual multi-tool orchestration

In [3]:
messages = [
    HumanMessage(
        "Which will be my projected bill? I'm in San Francisco, and I have oven. My past bill was $45"
    )
]

llm_with_tools = llm.bind_tools([get_weather, get_projection_bill])

while True:
    ai_message = llm_with_tools.invoke(messages)
    print("\nAI response content:", ai_message.content)
    tool_calls = getattr(ai_message, "tool_calls", None)
    print("Tool calls:", tool_calls)

    if not tool_calls:
        print("\nNo tool calls detected — conversation finished.")
        final_message = ai_message
        break

    messages.append(ai_message)

    for tool_call in tool_calls:
        tool_name = tool_call["name"].lower()
        selected_tool = tool_map.get(tool_name)
        if not selected_tool:
            print(f"Unknown tool requested: {tool_name}")
            continue
        print(f"Executing tool: {tool_name} with args: {tool_call.get('args', {})}")
        tool_msg = selected_tool.invoke(tool_call)
        print("Tool result:", tool_msg.content)
        messages.append(tool_msg)

print("\n************************ Conversation Ended ************************")
print("Final model message:\n", final_message.content)



AI response content: [{'id': 'rs_038f5c95ad9168f201690e9355ee048197bfb6bef6496a293a', 'summary': [{'text': '**Clarifying billing details**\n\nI notice we have tools: get_weather(city) and get_projection_bill({current_bill: integer, gas_oven: boolean}). The user\'s question about their projected bill in San Francisco points to using the projection tool with a current bill of $45 and a gas oven. But I need clarification since “oven” could mean gas or electric. It’s better to ask, "Is your oven gas or electric?" Maybe I could also ask about other appliances, but I need the boolean clarification first!', 'type': 'summary_text'}, {'text': '**Requesting clarification on oven type**\n\nI think it’s best to ask a clarifying question about the oven type. While I could offer projections for both gas and electric scenarios, the tool needs a boolean value. Calling it twice might lead to errors, so I’ll stick with just one question for accuracy. I’ll ask, "Is your oven gas or electric?" Once I get

## Automatic (Agent) multi-step orchestration

In [4]:
agent = create_agent(
    llm,
    tools=[get_weather, get_projection_bill],
    system_prompt="use one or more tools to get an answer; reply in Hinglish (Hindi mixed with English)"
)

messages_agent = [
    HumanMessage(
        content="Which will be my projected bill? I'm in San Francisco, and I have oven. My past bill was $45"
    )
]

print("************************** Agent Multi-Step invoke and details **************************")
response = agent.invoke({"messages": messages_agent})
pretty_print(response)

print("\n************************** Agent Multi-Step stream **************************")
for chunk in agent.invoke({"messages": messages_agent}, stream_mode="updates"):
    for step, data in chunk.items():
        print(f"step: {step}", flush=True)
        print(f"content: {data['messages'][-1].content_blocks}", flush=True)


************************** Agent Multi-Step invoke and details **************************
 1. [HUMAN] Which will be my projected bill? I'm in San Francisco, and I have oven. My past bill was $45
 2. [AI] [{'id': 'rs_0de693112157a4f501690e93ec4a448195b8c4cf10bbab2bb6', 'summary': [{'text': '**Clarifying tool usage**\n\nI see that I need to use tools, specifically get_weather and get_projection_bill. The user has mentioned their past bill of $45 and that they have an oven, but it’s not clear whether it\'s electric or gas. Since the function for projecting bills expects a boolean for a gas oven, I wonder if I should clarify that first. Though the instructions say to use the tools right away, I think asking, "Kya aapka oven gas hai ya electric?" in Hinglish would help before I proceed.', 'type': 'summary_text'}, {'text': '**Asking for clarification**\n\nI think it might be helpful to run parallel tools, but maybe not necessary for the weather. Before proceeding, I should definitely ask a f